In [1]:
# init
import datetime as dt
import json
import multiprocessing as mul
import os
from pathlib import Path
import re
import requests
import sys

from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as figf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import synapseclient
from synapseclient import Activity, Entity, Project, Folder, File
import synapseutils
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
import xmltodict

sys.path.insert(0, os.path.abspath('..'))

import utils as utils

data_dir = os.path.join(Path.home(), '.mha')
init_notebook_mode(connected=True)
InteractiveShell.ast_node_interactivity = 'all'
tqdm.pandas(leave=True)

metaData = 'syn15666940'
syn = synapseclient.Synapse()
syn.login()

raw = syn.store(Folder(
    name='raw',
    parent=metaData
))

def evaluation(df, exclusions=[]):
    """display unique values and bincounts

    Args:
        dataframe: (DataFrame) Pandas DataFrame to process
        exclusions: [(str)] optional list of columns to exclude
    """
    cols = list(set(df.columns) - set(exclusions)) if exclusions is not None else df
    row_sep = '-'.join(["-" for i in range(15)])
    for col in cols:
        print(col)
        print(row_sep)
        print(f'{df[col].value_counts(dropna=False)}\n')

    print("excluded NaN counts")
    print(row_sep)
    for col in exclusions:
        print(f'{col} - {len(df[df[col].isnull()])}')
        
def nan_percentages(df):
    cols = df.columns
    d, p = len(df), {}
    
    for col in cols:
        a = len(df.loc[df[col].isna()])
        p[col] = a/d
    
    print('{:<20} {:<5}'.format('col', 'perc_nan'))
    for k, v in p.items():
        n = len(k) if len(k) <= 20 else 20
        print('{:<20} {:<5}'.format(k[:n], np.round(v, 4)))

def isnum(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

def iqr(a):
    x = np.percentile(a, [25, 75])
    return x[1] - x[0]

def dpath(s):
    return os.path.join(data_dir, s)

Welcome, Luke Waninger!



In [2]:
cpu_count = mul.cpu_count()
print(f'WARNING: This notebook will consume {cpu_count} of {mul.cpu_count()} cpu cores.')

## Data Load

### Race
https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

In [3]:
rname = 'ACS_16_5YR_race.csv'

race_data = syn.store(File(
        path=dpath(rname),
        name=rname,
        parent=raw))

race = pd.read_csv(dpath(rname))
race['state']  = race.county.apply(lambda x: x[x.find(',')+1:].strip())
race['county'] = race.county.apply(lambda x: re.sub(r'County', '', x[:x.find(',')]).strip())

race.head(); len(race)

,GEO.id,GEO.id2,county,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,...,HD02_VD17,HD01_VD18,HD02_VD18,HD01_VD19,HD02_VD19,HD01_VD20,HD02_VD20,HD01_VD21,HD02_VD21,state
0,0500000US01001,1001,Autauga,55049,*****,53633,*****,41663,161,10113,...,29,641,358,19,30,19,30,0,27,Alabama
1,0500000US01003,1003,Baldwin,199510,*****,190798,*****,165950,245,18406,...,27,1461,551,432,244,333,230,99,81,Alabama
2,0500000US01005,1005,Barbour,26614,*****,25467,*****,12212,154,12745,...,21,901,135,6,10,6,10,0,21,Alabama
3,0500000US01007,1007,Bibb,22572,*****,22070,*****,16876,21,4788,...,21,8,15,0,21,0,21,0,21,Alabama
4,0500000US01009,1009,Blount,57704,*****,52668,*****,50582,147,899,...,27,371,212,138,120,23,27,115,116,Alabama


3142

### Income
https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml

In [4]:
iname = 'ACS_16_5YR_income.csv'

income_data = syn.store(File(
    path=dpath(iname),
    name=iname,
    parent=raw
))

income = pd.read_csv(dpath(iname))
income['state']  = income.county.progress_apply(lambda x: x[x.find(',')+1:].strip())
income['county'] = income.county.progress_apply(lambda x: re.sub(r'County', '', x[:x.find(',')]).strip())
income = income.loc[:, ['state', 'county', 'HC02_EST_VC02']]
income.columns = ['state', 'county', 'median_household_income']

income.head()

100%|██████████| 3142/3142 [00:00<00:00, 240584.61it/s]


,state,county,median_household_income
0,Alabama,Autauga,53099
1,Alabama,Baldwin,51365
2,Alabama,Barbour,33956
3,Alabama,Bibb,39776
4,Alabama,Blount,46212


### Zipcodes
We need a base zipcode file that includes the zip, state, county, latitude, and longitude

#### Get the national zip code listing from American Fact Finder

In [7]:
zname = 'zbp16totals.csv'
zips = pd.read_csv(dpath(zname))
zips_syn = syn.store(File(
    path=dpath(zname),
    name=zname,
    parent=raw
))

zips.drop(columns=['name'], inplace=True)

zips.city     = zips.city.apply(str.lower)
zips.stabbr   = zips.stabbr.apply(str.lower)
zips.cty_name = zips.cty_name.apply(str.lower)

flag_counts = {
    'empflag':zips.empflag.value_counts(),
    'emp_nf': zips.emp_nf.value_counts(),
    'qp1_nf': zips.qp1_nf.value_counts(),
    'ap_nf' : zips.ap_nf.value_counts()
}

zips.drop(columns=[
    'empflag', 'emp_nf', 'qp1_nf', 'ap_nf', 
    'qp1', 'ap', 'est', 'emp'
], inplace=True)

zips.columns = [
    'zip_code', 'city', 'state', 'county'
]

zips.head()
del zname, flag_counts

,zip_code,city,state,county
0,501,holtsville,ny,suffolk
1,1001,agawam,ma,hampden
2,1002,amherst,ma,hampshire
3,1003,amherst,ma,hampshire
4,1004,amherst,ma,hampshire


### Urban Areas
https://www.census.gov/geo/reference/ua/urban-rural-2010.html

In [9]:
ua_name = 'ua_list_all.csv'
ua_syn = syn.store(File(
    path=dpath(ua_name),
    name=ua_name,
    parent=raw
))
ualist = pd.read_csv(dpath(ua_name))

ualist.columns  = [c.strip().lower() for c in ualist.columns]
ualist['state'] = [x[x.find(',')+1:].strip().lower() for x in ualist.name]
ualist['city']  = [x[:x.find(',')].strip().lower() for x in ualist.name]
ualist = ualist.drop(columns='name')

ualist.head()

,uace,pop,hu,arealand,arealandsqmi,areawater,areawatersqmi,popden,lsadc,state,city
0,37,19824,8460,29222871,11.28,300497,0.12,1757.0,76,la,abbeville
1,64,5243,2578,11315197,4.37,19786,0.01,1200.1,76,sc,abbeville
2,91,3966,1616,5363441,2.07,13221,0.01,1915.2,76,wi,abbotsford
3,118,4666,2050,7416616,2.86,52732,0.02,1629.4,76,ms,aberdeen
4,145,25977,12114,33002447,12.74,247597,0.10,2038.6,76,sd,aberdeen


## Feature Generation

### Race

In [10]:
def topn(ri):
    cols = [i for i in range(3, len(ri))]

    maj_idx  = ri.iloc[cols].values.argmax()
    majority = ri.index[cols][maj_idx]
    majority_prop = np.round(ri.loc[majority]/ri.total, 3)
    
    cols = [c for c in cols if c != maj_idx+3]

    min_idx  = ri.iloc[cols].values.argmax()
    minority = ri.index[cols][min_idx]
    minority_prop = np.round(ri.loc[minority]/ri.total, 3) if ri.loc[minority] != 0 else 0
    
    return dict(
        county=ri.county,
        state=ri.state,
        race_majority=majority, 
        race_majority_prop=majority_prop,
        race_minority=minority, 
        race_minority_prop=minority_prop
    )


ccols = ['county', 'state']+[f'HD01_VD0{i}' for i in range(1, 10)]
race = race.loc[:, ccols]

ccols =  [
    'total', 'not_hispanic', 'white', 'black_or_african_american', 
    'american_indian', 'asian', 'pacific_islander', 'something_else',
    'two_plus'
]
race.columns = ['county', 'state'] + ccols

race['hispanic'] = race.total - race.not_hispanic
race.drop(columns=['not_hispanic'], inplace=True)

race = pd.merge(race, pd.DataFrame(list(race.progress_apply(topn, axis=1))), on=['state', 'county'])
race.drop(columns=[c for c in ccols if c != 'not_hispanic']+['hispanic'], inplace=True)

race.head()
features = race.copy(); del race

100%|██████████| 3142/3142 [00:01<00:00, 2554.70it/s]


,county,state,race_majority,race_majority_prop,race_minority,race_minority_prop
0,Autauga,Alabama,white,0.757,black_or_african_american,0.184
1,Baldwin,Alabama,white,0.832,black_or_african_american,0.092
2,Barbour,Alabama,black_or_african_american,0.479,white,0.459
3,Bibb,Alabama,white,0.748,black_or_african_american,0.212
4,Blount,Alabama,white,0.877,hispanic,0.087


### Income

In [11]:
features = pd.merge(features, income, on=['state', 'county'])
features.head()

del income

,county,state,race_majority,race_majority_prop,race_minority,race_minority_prop,median_household_income
0,Autauga,Alabama,white,0.757,black_or_african_american,0.184,53099
1,Baldwin,Alabama,white,0.832,black_or_african_american,0.092,51365
2,Barbour,Alabama,black_or_african_american,0.479,white,0.459,33956
3,Bibb,Alabama,white,0.748,black_or_african_american,0.212,39776
4,Blount,Alabama,white,0.877,hispanic,0.087,46212


### Walk scores
https://www.walkscore.com/

In [12]:
def load_page_from_disk(z):
    wpath = os.path.join(data_dir, 'walkscores', f'{str(z)}.txt')
    
    try:
        with open(wpath, 'r') as f:
            page = str(f.read())   
            
        return z, page
    except FileNotFoundError:
        return z, 'file not found'
    
    
def request_page(z):
    url = f'https://www.walkscore.com/score/{z}'
    response = requests.get(url)
    
    if response.ok:
        wpath = os.path.join(data_dir, 'walkscores', f'{str(z)}.txt')
        
        with open(wpath, 'w+') as f:
            f.write(str(response.content))
            
        return z, str(response.content)
    else:
        return z, str(response.status_code)
    
    
def walk_score(s):
    i = s.find('/badge/walk/score/')+18
    score = s[i:i+2]
    return score


# first try to load what we can from disk
pool  = mul.Pool(cpu_count)
pages = list(pool.map(load_page_from_disk, zips.zip_code))

# query any not cached
vals   = [p[0] for p in pages if p[1] == 'file not found']
pages += list(pool.map(request_page, vals))

# scrape the walk score from each html file
scores = pd.DataFrame([dict(zip_code=tup[0], score=walk_score(tup[1])) for tup in pages])
scores.columns = ['walk_score', 'zip_code']

# convert scores to numbers
mask = scores.walk_score.apply(isnum)
scores.loc[ mask, 'walk_score'] = pd.to_numeric(scores.loc[mask].walk_score)
scores.loc[~mask, 'walk_score'] = np.nan

# merge with main zip file
zips = zips.merge(scores, on='zip_code', how='outer')
zips.head()

pool.close(); pool.join()
del pool, pages, vals, scores, mask

,zip_code,city,state,county,walk_score
0,501,holtsville,ny,suffolk,1
1,1001,agawam,ma,hampden,91
2,1002,amherst,ma,hampshire,3
3,1003,amherst,ma,hampshire,77
4,1004,amherst,ma,hampshire,87


### Urban vs. Non-urban

In [13]:
def ua_designation(args):
    city, state = args
    
    r = ualist.loc[(ualist.city == city) & (ualist.state == state)]
    d = 'rural'
    
    if len(r) > 0:
        d = r.iloc[0].lsadc
        d = 'urban_area' if d == 75 else 'urban_cluster'

    return d

pool = mul.Pool(cpu_count)
zips['geo_designation'] = list(pool.map(ua_designation, [(r.city, r.state) for r in zips.itertuples()]))
zips.head()

pool.close(); pool.join()
del pool, ualist

,zip_code,city,state,county,walk_score,geo_designation
0,501,holtsville,ny,suffolk,1,rural
1,1001,agawam,ma,hampden,91,rural
2,1002,amherst,ma,hampshire,3,rural
3,1003,amherst,ma,hampshire,77,rural
4,1004,amherst,ma,hampshire,87,rural


## Final files

#### Zipcodes

In [14]:
sname = 'state_codes.csv'
state_abbr = syn.store(File(
    path=dpath(sname),
    name=sname,
    parent=raw
))
states = pd.read_csv(dpath(sname))

states.State = states.State.apply(str.lower)
states.Code  = states.Code.apply(str.lower)

zips = pd.merge(zips, states, left_on=['state'], right_on='Code', how='inner')
zips = zips.drop(columns=['Code'])
zips.columns = list(zips.columns.values)[:-1]+['state_name']

zips.city   = zips.city.apply(str.title)
zips.county = zips.county.apply(str.title)
zips.state  = zips.state.apply(str.upper)
zips.state_name = zips.state_name.apply(str.title)

zname = 'MHA_zipcode_metadata.csv'
zips.to_csv(dpath(zname), index=None)

MHA_zipcode_metadata = syn.setProvenance(
    syn.store(File(name=zname, path=dpath(zname), parent=metaData)),
    activity=Activity(
        name=zname,
        description='A csv file containing features for each zipcode.',
        used=[
            state_abbr, zips_syn, ua_syn,
            dict(
                name='County Business Patterns',
                url='https://www.census.gov/programs-surveys/cbp.html',
            )
        ]
    )
)


##################################################
 Uploading file to Synapse storage 
##################################################



#### Counties

In [15]:
fname = 'MHA_county_metadata.csv'
features.to_csv(dpath(fname), index=None)

features_syn = syn.store(File(
    path=dpath(fname),
    name=fname,
    parent=metaData
))

features_syn = syn.setProvenance(
    features_syn,
    activity=Activity(
        name='Process census microdata',
        description='Processes microdata from IPUM census files into county level feature aggregations',
        used=[
            state_abbr, race_data, income_data, zips_syn, ua_syn,
            dict(
                name='American FactFinder',
                url='https://factfinder.census.gov/faces/nav/jsf/pages/index.xhtml',
            ),
            dict(
                name='County Business Patterns',
                url='https://www.census.gov/programs-surveys/cbp.html',
            )
        ]
    )
)


##################################################
 Uploading file to Synapse storage 
##################################################



In [16]:
nan_percentages(zips)
len(zips)

iplot(go.Figure([go.Histogram(x=zips.walk_score)]))
iplot(go.Figure([go.Bar(
    x=pd.unique(zips.geo_designation), 
    y=pd.value_counts(zips.geo_designation),
    text=pd.value_counts(zips.geo_designation),
    textposition='outside'
)]))

col                  perc_nan
zip_code             0.0  
city                 0.0  
state                0.0  
county               0.0  
walk_score           0.0048
geo_designation      0.0  
state_name           0.0  


38812

In [17]:
nan_percentages(features)
len(features)

iplot(go.Figure([
    go.Box(name='majority', y=features.race_majority_prop),
    go.Box(name='minority', y=features.race_minority_prop)
]))

iplot(go.Figure([
    go.Bar(name='majority', x=pd.unique(features.race_majority), y=pd.value_counts(features.race_majority)),
    go.Bar(name='minority', x=pd.unique(features.race_minority), y=pd.value_counts(features.race_minority))
]))

col                  perc_nan
county               0.0  
state                0.0  
race_majority        0.0  
race_majority_prop   0.0  
race_minority        0.0  
race_minority_prop   0.0  
median_household_inc 0.0  


3142